In [4]:
import ee, eemont
#ee.Authenticate()
ee.Initialize()

In [5]:
import geemap
from importlib import reload  

In [6]:
from modules import high_level_functions
from modules import utils_Landsat_SR as utils_LS
from modules import ms_indices as indices
from modules import configs, utils_string

In [7]:
utils_string = reload(utils_string)

In [83]:
# PROPERTIES
# SET METADATA PARAMETERS
MAXCLOUD = 70
STARTYEAR = 2001
ENDYEAR = 2020
STARTMONTH = 7
ENDMONTH = 8
SCALE = 30
longitudes = [10]
latitudes = [45]
SIZE_LON = 10
SIZE_LAT = 5

target_collection = 'users/ingmarnitze/TCTrend_SR_2001-2020_TCVIS'
target_collection_nObs = 'users/ingmarnitze/TCTrend_SR_2001-2020_nObservations'

In [84]:
# image metadata Filters
config_trend = {
  'date_filter_yr' : ee.Filter.calendarRange(STARTYEAR, ENDYEAR, 'year'),
  'date_filter_mth' : ee.Filter.calendarRange(STARTMONTH, ENDMONTH, 'month'),
  'meta_filter_cld' : ee.Filter.lt('CLOUD_COVER', MAXCLOUD),
  'select_bands_visible' : ["B1", "B2","B3","B4"],
  'select_indices' : ["TCB", "TCG", "TCW"],
  'select_TCtrend_bands' : ["TCB_slope", "TCG_slope", "TCW_slope"],
  'geom' : None
}
#------ RUN FULL PROCESS FOR ALL REGIONS IN LOOP ------------------------------
#Map.addLayer(imageCollection, {}, 'TCVIS')

In [85]:
RUN = 1

In [86]:
for lowLat in latitudes:
    for leftLon in longitudes:
        
        
        # check for Hemisphere
        if lowLat < 0:
            sizeLat = SIZE_LAT * -1
        else:
            sizeLat = SIZE_LAT
            
        sizeLon = SIZE_LON
        
        # create Bounding Box
        geom = ee.Geometry.Polygon([leftLon,lowLat+sizeLat, leftLon, lowLat, leftLon+sizeLon, lowLat, leftLon+sizeLon, lowLat+sizeLat])
        config_trend['geom'] = geom
        Map.addLayer(geom,{}, str(lowLat))

        assetname_new = utils_string.make_TCTrendAssetNameSR(leftLon, lowLat, STARTYEAR, ENDYEAR)
        assetname_nObs = assetname_new + '_nObservations'

        # Calculate Trend
        trend = high_level_functions.runTCTrend(config_trend)
        if RUN:
            task = ee.batch.Export.image.toAsset(
                image=trend['visual'],
                description=assetname_new,
                assetId=target_collection + '/' + assetname_new,
                scale=SCALE,
                region=geom,
                maxPixels=1e12)

            task.start()

            task = ee.batch.Export.image.toAsset(
                image=trend['n_observations'],
                description=assetname_nObs,
                assetId=target_collection_nObs + '/' + assetname_nObs,
                scale=SCALE,
                region=geom,
                maxPixels=1e12)

            task.start()


In [87]:
task.status()

{'state': 'READY',
 'description': 'TCTrend_SR_E010N45_2001-2020_TCVIS_nObservations',
 'creation_timestamp_ms': 1632991788477,
 'update_timestamp_ms': 1632991788477,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'WFQBFYDDBAYUJM4BTD4F775E',
 'name': 'projects/earthengine-legacy/operations/WFQBFYDDBAYUJM4BTD4F775E'}

In [39]:
Map = geemap.Map()
Map.addLayer(geom)

In [50]:
Map.addLayer(ee.ImageCollection(target_collection[:-1]))

In [54]:
Map.add_basemap(basemap='SATELLITE')

In [67]:
Map.addLayer(geom,{}, str(lowLat))

In [68]:
Map

Map(bottom=19624.0, center=[-30.713503990354965, 20.978095751290322], controls=(WidgetControl(options=['positi…